In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from scipy import stats
from sklearn import datasets
from IPython.display import display, HTML

In [2]:
pip install ISLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.

In [3]:
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
(LinearDiscriminantAnalysis as LDA,
QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [34]:
from sklearn.metrics import confusion_matrix

# 5.5 Applied Exercises

## Question 5 - In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [57]:
Default = load_data('Default')

### (a) - Fit a logistic regression model that uses income and balance to predict default.

In [58]:
Default.student

0        No
1       Yes
2        No
3        No
4        No
       ... 
9995     No
9996     No
9997     No
9998     No
9999    Yes
Name: student, Length: 10000, dtype: category
Categories (2, object): ['No', 'Yes']

In [59]:
encoding = {'Yes': 1, 'No': 0}
Default['default'] = Default['default'].map(encoding)
Default['student'] = Default['student'].map(encoding)
print(Default.student, Default.default)

0       0
1       1
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    1
Name: student, Length: 10000, dtype: category
Categories (2, int64): [0, 1] 0       0
1       0
2       0
3       0
4       0
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: default, Length: 10000, dtype: category
Categories (2, int64): [0, 1]


In [25]:
from sklearn.metrics import accuracy_score
Predictors = ['income', 'balance']
lr = LogisticRegression()
X = Default[Predictors]
y = Default['default']
pred = lr.fit(X,y).predict(Default[Predictors])
accuracy_score(pred, Default['default'])


0.9737

In [26]:
def confusion_table(confusion_mtx):
    """Renders a nice confusion table with labels"""
    confusion_df = pd.DataFrame({'y_pred=0': np.append(confusion_mtx[:, 0], confusion_mtx.sum(axis=0)[0]),
                                 'y_pred=1': np.append(confusion_mtx[:, 1], confusion_mtx.sum(axis=0)[1]),
                                 'Total': np.append(confusion_mtx.sum(axis=1), ''),
                                 '': ['y=0', 'y=1', 'Total']}).set_index('')
    return confusion_df

def total_error_rate(confusion_matrix):
    """Derive total error rate from confusion matrix"""
    return 1 - np.trace(confusion_mtx) / np.sum(confusion_mtx)

###(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

>i. Split the sample set into a training set and a validation set.

>ii. Fit a multiple logistic regression model using only the training observations.

>iii. Obtain a prediction of default status for each individual in
the validation set by computing the posterior probability of
default for that individual, and classifying the individual to
the default category if the posterior probability is greater
than 0.5.

>iv. Compute the validation set error, which is the fraction of
the observations in the validation set that are misclassifed.

### (c) Repeat the process in (b) three times, using three diferent splits of the observations into a training set and a validation set. Comment on the results obtained.

In [37]:
pip install --upgrade pandas

In [38]:
Default

AttributeError: 'Index' object has no attribute '_format_flat'

     default student      balance        income
0          0      No   729.526495  44361.625074
1          0     Yes   817.180407  12106.134700
2          0      No  1073.549164  31767.138947
3          0      No   529.250605  35704.493935
4          0      No   785.655883  38463.495879
...      ...     ...          ...           ...
9995       0      No   711.555020  52992.378914
9996       0      No   757.962918  19660.721768
9997       0      No   845.411989  58636.156984
9998       0      No  1569.009053  36669.112365
9999       0     Yes   200.922183  16862.952321

[10000 rows x 4 columns]

In [64]:
def testError(n: int, xVars: list[str]) -> list[int]:
  ret = []
  for i in range(n):
    X_train, X_test, y_train, y_test = train_test_split(Default[xVars], Default['default'], test_size = .25, random_state = i)
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    pred = lr.predict(X_test)
    print(confusion_matrix(pred, y_test))
    ret.append(accuracy_score(pred, y_test))
  return ret

ret = testError(4, Predictors)
print(ret)



[[2403   96]
 [   1    0]]
[[2421   79]
 [   0    0]]
[[2432   67]
 [   1    0]]
[[2421   50]
 [   3   26]]
[0.9612, 0.9684, 0.9728, 0.9788]


### (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [65]:
xVar = ['income', 'balance', 'student']
ret2 = testError(4, xVar)
print(ret2)

[[2403   96]
 [   1    0]]
[[2421   79]
 [   0    0]]
[[2432   67]
 [   1    0]]
[[2420   50]
 [   4   26]]
[0.9612, 0.9684, 0.9728, 0.9784]


In [66]:
diff = []
for i in range(len(ret)):
  diff.append(ret[i] - ret2[i])
diff


[0.0, 0.0, 0.0, 0.00039999999999995595]

#### Results - (d)

As shown in the output list there is not a noticable difference between adding students and not adding students. Thus the variable seems to be ineffective with this type of modeling

## Question 6 - We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefcients in two diferent ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the sm.GLM() function. Do not forget to set a random seed before beginning your analysis.

### (a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefcients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [68]:
predictors = ['income', 'balance']
X = Default[predictors]
X = sm.add_constant(X)
y = Default['default']
results = sm.Logit(y,X).fit()
results.summary()


Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Thu, 27 Jun 2024   Pseudo R-squ.:                  0.4594
Time:                        16:52:56   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

### (b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefcient estimates for income and balance in the multiple logistic regression model.

In [98]:
def boot_fn(df, idx):
    response   = 'default'
    predictors = ['income', 'balance']

    X = sm.add_constant(np.array(df[predictors].loc[idx]));
    y = np.array(df[response].loc[idx])

    model_logit = sm.Logit(y, X).fit();
    print(model_logit.params)
    return model_logit.params;

### (c) - Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefcients for income and balance.

In [99]:
def boot_idx(n):
  return np.random.randint(low = 0, high = n, size = n)

In [100]:
def boot(fn, data_df, samples):
    """Perform bootstrap for B number of samples"""
    results = []
    for s in range(samples):
        Z = fn(data_df, boot_idx(data_df.shape[0]))
        results += [Z]
    return np.array(results)

In [103]:
def standard_deviation(X):
  np.std(X, axis=0)


In [104]:
arr = boot(boot_fn, Default, 1000)
pred = np.mean(arr, axis = 0)
std = standard_deviation(arr)
print('predictions are:', pred)
print('standard deviations are:', std)

Optimization terminated successfully.
         Current function value: 0.077692
         Iterations 10
[-1.15891357e+01  1.75912944e-05  5.77511964e-03]
Optimization terminated successfully.
         Current function value: 0.084263
         Iterations 10
[-1.09578850e+01  1.51619409e-05  5.41008539e-03]
Optimization terminated successfully.
         Current function value: 0.076623
         Iterations 10
[-1.22167856e+01  2.42761120e-05  5.98895804e-03]
Optimization terminated successfully.
         Current function value: 0.079868
         Iterations 10
[-1.13000280e+01  1.56455994e-05  5.61677106e-03]
Optimization terminated successfully.
         Current function value: 0.077689
         Iterations 10
[-1.17313265e+01  2.06241870e-05  5.79681526e-03]
Optimization terminated successfully.
         Current function value: 0.076040
         Iterations 10
[-1.22715525e+01  3.01956232e-05  5.89782790e-03]
Optimization terminated successfully.
         Current function value: 0.075039
  

### (d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

* I believe I somehow miss coded my function and their predictions for coeff and std should be close.

## Question 7 - In Sections 5.1.2 and 5.1.3, we saw that the cross_validate() function can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just sm.GLM() and the predict() method of the ftted model within a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the Weekly data set. Recall that in the context of classifcation problems, the LOOCV error is given in (5.4).